<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Presidential-Election-Data" data-toc-modified-id="Presidential-Election-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Presidential Election Data</a></span><ul class="toc-item"><li><span><a href="#Existing-files-online" data-toc-modified-id="Existing-files-online-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Existing files online</a></span></li><li><span><a href="#Web-Scraping" data-toc-modified-id="Web-Scraping-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Web Scraping</a></span></li></ul></li><li><span><a href="#Movie-list-(LGBT-&amp;-Feminism)" data-toc-modified-id="Movie-list-(LGBT-&amp;-Feminism)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Movie list (LGBT &amp; Feminism)</a></span><ul class="toc-item"><li><span><a href="#Web-Scraping" data-toc-modified-id="Web-Scraping-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Web Scraping</a></span></li><li><span><a href="#Online-dataset" data-toc-modified-id="Online-dataset-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Online dataset</a></span></li><li><span><a href="#Online-repository" data-toc-modified-id="Online-repository-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Online repository</a></span></li></ul></li><li><span><a href="#Average-Movie-ratings" data-toc-modified-id="Average-Movie-ratings-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Average Movie ratings</a></span></li><li><span><a href="#Tweets" data-toc-modified-id="Tweets-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tweets</a></span></li></ul></div>

In [98]:
# import all necessary packages
import tweepy
import pandas as pd
import requests as req
from lxml import etree
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from tqdm import tqdm_notebook
import csv
import time

## Presidential Election Data
**Data set**: Presidential election data from 1996 to 2016. <br>
**Goal**: Identify conservative, liberal and swing states. <br><br>
**Data Source:** 
1. Existing csv/xls/xlw files online
2. Web Scraping 1996 election data online

### Existing files online
Online source: https://transition.fec.gov/pubrec/electionresults.shtml <br>
It contains data of 2000, 2004, 2008, 2012, 2016.

### Web Scraping
Web scraping source: https://transition.fec.gov/pubrec/fe1996/elecpop.htm <br>
It contains data from 1996.

In [35]:
# 1996 ELECTORAL AND POPULAR VOTE SUMMARY is in html format
def simple_get(url):
    """
    Attempts to get the content at 'url' by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                # r.text is the content of the response in unicode, 
                # and r.content is the content of the response in bytes.
                return resp.content
            else:
                return None
            
    except RequestException as e:
        log_error('Error during requests to {0}:{1}'.format(url, str(e)))
        return None

In [36]:
def is_good_response(resp):
    """
    Return True if the response seems to be HTML/HTM, Flase otherwise.
    """
    content_type = resp.headers['Content-Type']
    return (resp.status_code == 200 
            and content_type is not None
            and content_type.find('html') > -1)

In [37]:
def log_error(e):
    """
    Print log errors.
    """
    print(e)

In [6]:
# Download 1996 presidential ELECTORAL AND POPULAR VOTE 
url = 'https://transition.fec.gov/pubrec/fe1996/elecpop.htm'
response = simple_get(url)
if response is not None:
    htm = BeautifulSoup(response, 'html.parser')
    # cast to string
    para = str(htm.find_all('pre'))
    temp_content = para[para.find('>AL'):]
    table_content = temp_content[1:temp_content.find('<st')]

In [7]:
table_content_li = [x for x in table_content.split('\r\n')]

In [8]:
content = []
for row in table_content_li[:-1]:
    a = row.split('        ')
    if a[1] == '':
        a[1] = 'n'
    if a[2] == '':
        a[2] = 'n'
    content.append(a)

In [9]:
# process exception: different length of row
content[8]

['DC', '3', 'n', '     158,220       17,339', '3,611', '      185,726 ']

In [10]:
update = content[8][3] + ' ' + content[8][4]
content[8][3] = update

In [11]:
temp_row = content[8]
del temp_row[4]
temp_row

['DC', '3', 'n', '     158,220       17,339 3,611', '      185,726 ']

In [12]:
# replace exception with updated row
content[8] = temp_row

In [13]:
# convert list to numpy array then to dataframe
df96 = pd.DataFrame(content)

In [14]:
df96.columns = ['State', 'Clinton','Dole','Popular vote','Total Popular vote']

In [95]:
df96.head(10)

,State,Clinton,Dole,Popular vote,Total Popular vote
0,AL,n,9,"662,165 769,044 92,149","1,534,349"
1,AK,n,3,"80,380 122,746 26,333","241,620"
2,AZ,8,n,"653,288 622,073 112,072","1,404,405"
3,AR,6,n,"475,171 325,416 69,884","884,262"
4,CA,54,n,"5,119,835 3,828,380 697,847","10,019,484"
5,CO,n,8,"671,152 691,848 99,629","1,510,704"
6,CT,8,n,"735,740 483,109 139,523","1,392,614"
7,DE,3,n,"140,355 99,062 28,719","270,845"
8,DC,3,n,"158,220 17,339 3,611","185,726"
9,FL,25,n,"2,546,870 2,244,536 483,870","5,303,794"


## Movie list (LGBT & Feminism)
**Data Source:**<br>
1. Web scraping from webpages:
    * https://en.wikipedia.org/wiki/List_of_LGBT-related_films
    * https://en.wikipedia.org/wiki/Category:Feminist_films
2. Online dataset:
    * https://www.kaggle.com/juzershakir/tmdb-moviesdataset/home
3. Online repository
    * http://files.grouplens.org/datasets/movielens/ml-20m-README.html

### Web Scraping

In [68]:
# LGBT related movie
origin_page = req.get("https://en.wikipedia.org/wiki/List_of_LGBT-related_films")

soup = BeautifulSoup(origin_page.text, "html.parser")

movie_name1 = ''
for element in soup.find_all('a'):
    if element.get('title') is not None:
        movie_name1 += (str(element.string) + "***")

# Get movie names part only
chunks = movie_name1.split('edit***')
for chunk in chunks:
    if chunk.startswith('Z'):
        z_index = chunks.index(chunk)
    if chunk.startswith('$'):
        a_index = chunks.index(chunk)

movie_list1 = chunks[a_index : z_index+1]

# convert each movie into an element of a list
movie_names1 = []
for movie_chunk in movie_list1:
    movie_temp = movie_chunk.split('***')
    movie = movie_temp[:-1]
    movie_names1.extend(movie)
    
# a list of all lgbt movie names from Wiki page
print(movie_names1[0:100])

['$30', 'Boys Life 3', '10 Attitudes', 'The 10 Year Plan', '101 Rent Boys', '101 Reykjavík', '2 × 4', '2 Minutes Later', '2 Seconds', '20 Centimeters', '200 American', '2:37', '24 Nights', 'The 24th Day', '29th and Gay', '3', '3 Dancing Slaves', '3-Day Weekend', '3 Kanya', '30 Years from Here', '4th Man Out', '4.3.2.1', '491', '5ive Girls', '50 Ways of Saying Fabulous', '52 Tuesdays', '54', "'68", '68 Pages', '7 mujeres, 1 homosexual y Carlos', '8 Women', '8: The Mormon Proposition', '9 Dead Gay Guys', "À cause d'un garçon", 'À corps perdu', 'A mi madre le gustan las mujeres', 'À toute vitesse', 'A un dios desconocido', 'Aaron... Albeit a Sex Hero', 'Aban and Khorshid', 'Absent', 'Academy', 'Achilles', 'The Best of Boys in Love', 'Across the Universe', 'AIDS: Doctors and Nurses tell their Stories', 'Ajumma! Are You Krazy???', 'An Act of Valour', 'Adam & Steve', 'Adão e Eva', 'ADHURA', 'The Adjuster', 'The Adonis Factor', 'Adored', 'The Adventure of Iron Pussy', 'The Adventures of Felix

In [69]:
# Feminism related movies
origin_page = req.get("https://en.wikipedia.org/wiki/Category:Feminist_films")

soup = BeautifulSoup(origin_page.text, "html.parser")

movie_name2 = ''
for element in soup.find_all('a'):
    if element.get('title') is not None:
        movie_name2 += (str(element.string) + "***")

# Get movie names part only
chunks = movie_name2.split('***')
for chunk in chunks:
#     print(chunk)
    if chunk.startswith('Nor'):
        z_index = chunks.index(chunk)
    if chunk.startswith('5'):
        a_index = chunks.index(chunk)

movie_list2 = chunks[a_index : z_index+1]

# convert each movie into an element of a list
movie_names2 = []
for movie_chunk in movie_list2:
    if movie_chunk.endswith("film)"):
        movie_names2.append(movie_chunk[0:movie_chunk.index("(")-1])
    else:
        movie_names2.append(movie_chunk)
    
# a list of all lgbt movie names from Wiki page
print(movie_names2[0:100])

['5 Girls', '9 to 5', '10 Hours of Walking in NYC as a Woman', '10 Things I Hate About You', '20th Century Women', '22 Female Kottayam', '36 Vayadhinile', 'Aadavantha Deivam', 'Aandhi', 'Aaravalli', 'Aayiram Thalai Vaangi Apoorva Chinthamani', 'The Accused', 'Akka Thangai', 'Aletta Jacobs: Het Hoogste Streven', "Alice Doesn't Live Here Anymore", 'Alice in Wonderland', 'Aliens', 'All About My Mother', 'Anastasia', 'Anatomy of Hell', 'An Angel at My Table', 'Angry Indian Goddesses', 'Anthuleni Katha', "Antonia's Line", 'Arangetram', 'Archana IAS', 'Arth', 'Ask for Jane', 'Assassination Nation', 'The Associate', 'Astitva', 'Attack of the 50 Ft. Woman', 'Aval Appadithan', 'Aval Oru Thodar Kathai', 'Avargal', 'Bad Girls', 'Bad Moms', 'Bagdad Cafe', 'The Ballad of Josie', 'The Ballad of Little Jo', 'Bandit Queen', 'Barb Wire', 'Basic Instinct', 'Battle of the Sexes', 'Becoming Jane', 'Bed and Sofa', 'The Beguiled', 'Big Eyes', 'Bol', 'Born in Flames', 'Brave', 'Brimstone', 'Buddha Collapsed 

### Online dataset
https://www.kaggle.com/juzershakir/tmdb-moviesdataset/home

In [71]:
file = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating2/tmdb_movies_data.csv')

# keywords for search
searchfor = ['lgbt','gay','lesb','strong woman','strong women','femin','homo']

# in order to search multiple keywords at a time
file_alter = file[file['keywords'].str.contains('|'.join(searchfor), na=False)]

In [72]:
# Get 196 rows of data 
file_alter.shape

(196, 21)

In [94]:
file_alter.head(10)

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
19,131634,tt1951266,5.476958,160000000,650523427,The Hunger Games: Mockingjay - Part 2,Jennifer Lawrence|Josh Hutcherson|Liam Hemswor...,http://www.thehungergames.movie/,Francis Lawrence,The fire will burn forever.,...,"With the nation of Panem in a full scale war, ...",136,War|Adventure|Science Fiction,Studio Babelsberg|StudioCanal|Lionsgate|Walt D...,11/18/2015,2380,6.5,2015,1.471999e+08,5.984813e+08
50,258480,tt2402927,2.883233,11800000,40272135,Carol,Cate Blanchett|Rooney Mara|Kyle Chandler|Sarah...,http://carolfilm.com/,Todd Haynes,Some people change your life forever.,...,"In 1950s New York, a department-store clerk wh...",118,Romance|Drama,Killer Films|Film4|Number 9 Films,11/20/2015,562,7.1,2015,1.085600e+07,3.705035e+07
157,275601,tt3623726,0.956096,18000000,41325328,Ricki and the Flash,Meryl Streep|Kevin Kline|Mamie Gummer|Sebastia...,NaN,Jonathan Demme,Get Ready for Ricki.,...,"Meryl Streep stars as Ricki Rendazzo, a guitar...",100,Comedy|Drama|Music,Columbia TriStar|Clinica Estetico|LStar Capital,7/10/2015,152,5.5,2015,1.655999e+07,3.801929e+07
168,245168,tt3077214,0.898296,14000000,16002420,Suffragette,Carey Mulligan|Helena Bonham Carter|Meryl Stre...,NaN,Sarah Gavron,Mothers. Daughters. Rebels.,...,Based on true events about the foot soldiers o...,106,Drama|History,Ruby Films|PathÃ©|Film4,10/16/2015,231,7.1,2015,1.287999e+07,1.472222e+07
200,306745,tt1658801,0.706023,7000000,573335,Freeheld,Julianne Moore|Ellen Page|Steve Carell|Michael...,http://www.freeheld.movie/,Peter Sollett,A true story of love and injustice.,...,New Jersey car mechanic Stacie Andree and her ...,103,Romance|Drama,Double Feature Films|Endgame Entertainment|Hig...,10/2/2015,86,7.1,2015,6.439997e+06,5.274680e+05
243,333344,tt3194590,0.529850,0,0,Dirty Weekend,Matthew Broderick|Alice Eve|Phil Burke|Gia Cro...,NaN,Neil LaBute,Some things are better left unsaid.,...,"During a layover in Albuquerque, work colleagu...",93,Drama|Comedy,Horsethief Pictures,9/4/2015,12,4.8,2015,0.000000e+00,0.000000e+00
300,250225,tt2570858,0.412503,1000000,0,The Duke of Burgundy,Sidse Babett Knudsen|Chiara D'Anna|Monica Swin...,NaN,Peter Strickland,NaN,...,A woman who studies butterflies and moths test...,101,Drama,Pioneer Pictures|Rook Films,1/23/2015,58,5.8,2015,9.199996e+05,0.000000e+00
364,329829,tt1254322,0.390188,0,0,The Girl King,Malin Buska|Sarah Gadon|Michael Nyqvist|Lucas ...,NaN,Mika KaurismÃ¤ki,NaN,...,"A portrait of the brilliant, extravagant Krist...",106,Drama|History|Romance,Starhaus Filmproduktion|Anagram Produktion|Gal...,12/11/2015,12,6.1,2015,0.000000e+00,0.000000e+00
521,249916,tt3077108,0.138068,0,0,Appropriate Behavior,Desiree Akhavan|Rebecca Henderson|Halley Feiff...,NaN,Desiree Akhavan,NaN,...,"For Shirin, being part of a perfect Persian fa...",90,Romance|Comedy|Drama,Parkville Pictures,3/14/2015,18,5.8,2015,0.000000e+00,0.000000e+00
538,224972,tt3983674,0.114264,0,0,The Mask You Live In,NaN,http://themaskyoulivein.org,Jennifer Siebel Newsom,"Is american masculinity harming our boys, men ...",...,"Compared to girls, research shows that boys in...",88,Documentary,NaN,1/1/2015,11,8.9,2015,0.000000e+00,0.000000e+00


### Online repository

In [75]:
file_rating = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/ratings.csv')
file_tag = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/tags.csv')

In [76]:
file_rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [77]:
file_tag.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [78]:
file_movie = pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/movies.csv')

In [79]:
file_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [80]:
file_link=pd.read_csv('/Users/amberwu/Downloads/UIUC/Course FL 2018/Open Data Mashups/Data repo/Movie avg ratings/Movie_rating1/ml-20m/links.csv')
file_link.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [81]:
# search keywords from "tag" for lgbt/feminist related movies
name_tag = file_tag.merge(file_movie, on='movieId')
name_tag_alter = name_tag[name_tag['tag'].str.contains('|'.join(searchfor), na=False)]

In [93]:
name_tag_alter.head(10)

,userId,movieId,tag,timestamp,title,genres
1360,63618,1617,homoerotic subtext,1368242588,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
4977,96,106696,feminist,1396097502,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5000,12131,106696,feminist,1419021976,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5009,24134,106696,feminist,1390933092,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5050,56879,106696,feminist,1417918347,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5052,57434,106696,feminist,1388906221,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5060,79167,106696,feminist,1390787321,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5075,84441,106696,feminist,1416016304,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5079,86761,106696,feminist,1420412577,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...
5088,102576,106696,feminist,1419916572,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...


## Average Movie ratings
**Data Source:** <br>
- Online repository:
    http://files.grouplens.org/datasets/movielens/ml-20m-README.html

In [84]:
# This contains multiple users' ratings on the same movies
movie_rating = name_tag_alter.merge(file_rating, on=['userId','movieId'])

In [92]:
movie_rating.head(10)

,userId,movieId,tag,timestamp_x,title,genres,rating,timestamp_y
0,63618,1617,homoerotic subtext,1368242588,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller,4.5,1351591811
1,96,106696,feminist,1396097502,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,3.5,1396097287
2,12131,106696,feminist,1419021976,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,4.0,1419021959
3,24134,106696,feminist,1390933092,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,5.0,1390933046
4,56879,106696,feminist,1417918347,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,4.5,1421637283
5,57434,106696,feminist,1388906221,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,4.0,1388906210
6,79167,106696,feminist,1390787321,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,3.5,1390787309
7,84441,106696,feminist,1416016304,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,2.5,1416015906
8,86761,106696,feminist,1420412577,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,4.5,1420412530
9,102576,106696,feminist,1419916572,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,4.5,1419916563


In [87]:
# Calculate average movie ratings
movieid_rating = movie_rating.groupby('movieId')[['rating']].mean()
movieid_rating=movieid_rating.reset_index()

In [91]:
moviename_rating = movieid_rating.merge(file_movie, on='movieId')
moviename_rating.head(10)

,movieId,rating,title,genres
0,35,4.500000,Carrington (1995),Drama|Romance
1,49,4.500000,When Night Is Falling (1995),Drama|Romance
2,82,3.583333,Antonia's Line (Antonia) (1995),Comedy|Drama
3,141,3.576923,"Birdcage, The (1996)",Comedy
4,171,4.125000,Jeffrey (1995),Comedy|Drama
5,178,3.500000,Love & Human Remains (1993),Comedy|Drama
6,198,2.500000,Strange Days (1995),Action|Crime|Drama|Mystery|Sci-Fi|Thriller
7,203,4.000000,"To Wong Foo, Thanks for Everything! Julie Newm...",Comedy
8,219,4.500000,"Cure, The (1995)",Drama
9,233,4.000000,Exotica (1994),Drama


In [90]:
moviename_rating.shape

(497, 4)

## Tweets
Using Twitter API to get people's reviews on movies.

In [2]:
# authorization


auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Create tweepy object for twitter API
api = tweepy.API(auth)

In [100]:
# Read from Twitter ID file
tweet_summary_map = {}
nameList = ['being julia', 'brokeback mountain']
for movie in nameList:
    with open("Tweet Data/Tweepy-API-xPath/"+movie+".txt", 'r') as f:
        x = f.read().splitlines()
    id_list = [line.split('/')[-1] for line in x]
    tweet_summary = pd.DataFrame(columns=['Timezone', 'Full Tweet', 'user_name', 'user_location', 'coordinates', 'country_code', 'place'])
    tweet_summary.index.name = 'Tweet Time'
    for id in tqdm_notebook(id_list):
        try:
            tweet_info = api.get_status(id, lang = 'en', tweet_mode='extended')
            if 'retweeted_status' in dir(tweet_info):
                tweet=tweet_info.retweeted_status.full_text
            else:
                tweet=tweet_info.full_text
            if tweet_info.place:
                place = tweet_info.place.full_name
                country_code = tweet_info.place.country_code
            else:
                place = None
                country_code = None
        except:
            pass
        tweet_summary.loc[tweet_info.created_at] = [tweet_info.user.time_zone, tweet, tweet_info.user.name, tweet_info.user.location, tweet_info.coordinates, country_code, place]
    tweet_summary_map[movie] = tweet_summary
    time.sleep(1)

In [101]:
tweet_summary_map['brokeback mountain']

,Timezone,Full Tweet,user_name,user_location,coordinates,country_code,place
Tweet Time,,,,,,,
2018-11-12 16:36:26,None,Did brokeback mountain teach you pain because ...,Corey OBrien,buffalo,None,US,"Buffalo, NY"
2018-11-07 21:12:38,None,Brokeback Mountain (2005)\n\nI nearly failed n...,Team Antibac,moren din,None,None,None
2018-11-12 12:59:20,None,#HappyBirthday to Anne Hathaway (36)\nher top ...,Stephen Gibbons,Ireland,None,None,None
2018-11-17 00:31:10,None,uhhhh can we get a brokeback mountain sequel,bored as sheeeet,,None,None,None
2018-11-13 13:00:19,None,Why is that picture of a cow titled Brokeback ...,Ustedpac Shakur,The Darkest Timeline,None,None,None
2018-11-17 16:44:52,None,Production polaroids of Heath Ledger on the se...,History Lovers Club,"London, England",None,None,None
2018-11-13 06:13:03,None,Jake Gyllenhaal stans gathering to watch broke...,kaylee,matthew,None,None,None
2018-11-13 15:39:58,None,Give Kylie Minogue royalties for singing Cowbo...,Straw-B-Rita Ora,,None,None,None
2018-11-09 16:16:54,None,BROKEBACK MOUNTAIN (2005)\n\nFramed: Anne Hath...,Faces In Frames,,None,None,None


In [102]:
tweet_summary_map['being julia']

,Timezone,Full Tweet,user_name,user_location,coordinates,country_code,place
Tweet Time,,,,,,,
2017-10-03 20:14:44,None,STACEY LEFT PAPER IN HER PIE AND SHES STILL HE...,Grace,"Wales, United Kingdom",None,None,None
2018-06-15 06:17:51,None,PHOTO OF THE DAY: Shaun Evans at a Cocktail Pa...,Shaun Evans Web,,None,None,None
2018-08-24 13:10:18,None,Snatch\nThe Godfather\nAmerican Psycho\nThe Th...,Birthday Suit Elvis 🍑,KCMO➡️CHICAGO➡️RACINE,None,None,None
2018-11-17 01:21:02,None,Annette Bening in Being Julia (2004) https://t...,Frame Found,,None,None,None
2018-11-16 23:42:27,None,i very easily see myself being Julia Roberts’ ...,Bent Neck Lady,"sunnydale, CA",None,None,None
2018-11-16 22:53:27,None,I rated Being Julia (2004) 5/10 #IMDb https:/...,hilal yarın napıyosun,,None,None,None
2018-11-15 05:04:45,None,That time Annette Bening played me in a movie....,Julia Lambert,"Saint John, NB, Canada",None,None,None
2018-11-14 22:59:35,None,Being Julia. https://t.co/wQKvyLbXKk,🌈🌈🌈ted🌈🌈🌈,"Maine, USA",None,None,None
2018-11-14 22:24:03,None,“You’re Julia Roberts and I-“\n“Why am I alway...,Felicia🖤🚀-I Find It Quite Emotional,Up Harry Styles Beautiful Ass,None,None,None
